In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct
print(ct.__file__)

plt.rcParams['figure.constrained_layout.use'] = True
%matplotlib widget

/home/avcopan/code/other/cantera-workshop/.pixi/envs/default/lib/python3.13/site-packages/cantera/__init__.py


## Mechanism files & basic info

https://github.com/Cantera/cantera/blob/main/data/gri30.yaml

In [2]:
gas = ct.Solution('gri30.yaml')

In [3]:
gas.n_species, gas.n_reactions

(53, 325)

In [4]:
print(gas.species_names)

['H2', 'H', 'O', 'O2', 'OH', 'H2O', 'HO2', 'H2O2', 'C', 'CH', 'CH2', 'CH2(S)', 'CH3', 'CH4', 'CO', 'CO2', 'HCO', 'CH2O', 'CH2OH', 'CH3O', 'CH3OH', 'C2H', 'C2H2', 'C2H3', 'C2H4', 'C2H5', 'C2H6', 'HCCO', 'CH2CO', 'HCCOH', 'N', 'NH', 'NH2', 'NH3', 'NNH', 'NO', 'NO2', 'N2O', 'HNO', 'CN', 'HCN', 'H2CN', 'HCNN', 'HCNO', 'HOCN', 'HNCO', 'NCO', 'N2', 'AR', 'C3H7', 'C3H8', 'CH2CHO', 'CH3CHO']


## Species thermo data

NASA 7-coefficient polynomials:

$$   \frac{\hat{c}_p^\circ(T)}{\overline{R}} = a_0 + a_1 T + a_2 T^2 + a_3 T^3 + a_4 T^4 $$

$$   \frac{\hat{h}^\circ (T)}{\overline{R} T} = a_0 + \frac{a_1}{2} T + \frac{a_2}{3} T^2 +
                         \frac{a_3}{4} T^3 + \frac{a_4}{5} T^4 + \frac{a_5}{T} $$

$$   \frac{\hat{s}^\circ(T)}{\overline{R}} = a_0 \ln T + a_1 T + \frac{a_2}{2} T^2 + \frac{a_3}{3} T^3 +
                      \frac{a_4}{4} T^4 + a_6 $$

In [ ]:
CH4 = gas.species("CH4")
CH4.thermo.input_data

In [ ]:
TT = np.linspace(300, 2000, 200)
cp = [CH4.thermo.cp(T) for T in TT]

In [ ]:
fig, ax = plt.subplots()
ax.plot(TT, cp);

In [ ]:
gas()

## Setting thermodynamic state
- Always requires two variables, e.g. $(T, P)$ or $(T, v)$ or $(s, P)$ etc.
- Specify `None` to hold a property constant

https://cantera.org/documentation/docs-3.0/sphinx/html/cython/thermo.html#thermophase

In [ ]:
gas.TP = 500, ct.one_atm
gas()

In [ ]:
gas.SP = None, 2 * ct.one_atm
gas()

## Setting composition
- Main ways are by setting either mass fractions ($Y$) or mole fractions ($X$)
- Can be set alone (in which case $(T,\rho)$ held constant), or with a valid property pair
- Mass/mole fractions are automatically normalized to 1.0
- Can also set equivalence ratio

In [ ]:
gas.TPX = 300, ct.one_atm, {'H2': 1.0, 'O2': 1.5}
gas()

In [ ]:
X = np.zeros(gas.n_species)
X[9:14] = 1.0
gas.X = X
gas()

In [ ]:
gas.TPY = None, None, "CH4:1.0, N2:3"
gas()

In [ ]:
gas.TP = 500, 10 * ct.one_atm
gas.set_equivalence_ratio(0.5, "CH4:1.0", "O2:1, N2:3.76")
gas()

In [ ]:
# gas.set_equivalence_ratio?

## Flame temperature calculation

With a guest appearance by the `SolutionArray` class.

In [ ]:
phis = np.linspace(0.4, 3.0, 100)
Tad = []
T0 = 300
P0 = ct.one_atm
fuel = "C2H6: 1.0"

for phi in phis:
    gas.TP = T0, P0
    gas.set_equivalence_ratio(phi, fuel, "O2:1.0, N2:3.76")
    gas.equilibrate("HP")
    Tad.append(gas.T)

In [ ]:
fig, ax = plt.subplots()
ax.plot(phis, Tad)
ax.set(xlabel="Equivalence ratio, $\\phi$", ylabel="adiabatic flame temperature [K]");

In [ ]:
states = ct.SolutionArray(gas, len(phis))
states.Y.shape

In [ ]:
states.TP = T0, P0
states.set_equivalence_ratio(phis, fuel, "O2:1.0, N2:3.76")
states.equilibrate("HP")

In [ ]:
fig, ax = plt.subplots()
species = ['CO2', 'CO', 'H2', 'O2']
ax.plot(phis, states(*species).X, label=species)
ax.legend();

## Kinetics & Reactions

https://cantera.org/documentation/docs-3.0/sphinx/html/cython/kinetics.html#kinetics

In [ ]:
R = gas.reaction(1)
R

In [ ]:
R.rate.pre_exponential_factor

In [ ]:
R.reactants

In [ ]:
gas.forward_rate_constants[1]

In [ ]:
gas.net_rates_of_progress[1]

## Defining reactions & exploring equilibrium

$$ a \mathrm{A} + b \mathrm{B} \rightleftharpoons c \mathrm{C} + d \mathrm{D} $$

$$ \Delta_r^\circ \hat{g} = \Delta_r^\circ \hat{h} - T \Delta_r^\circ \hat{s}$$

$$ K_c = \frac{k_f}{k_r} = \frac{[\mathrm{C}]^c [\mathrm{D}]^d}{[\mathrm{A}]^a [\mathrm{B}]^b} = \exp\left( \frac{\Delta_r^\circ \hat{g}}{RT} \right) \left( \frac{p^o}{RT}\right)^{\nu_{net}} $$

In [ ]:
R1 = ct.Reaction(equation="CO + O = CO2", rate=ct.Arrhenius(0.0, 0.0, 0.0))
R2 = ct.Reaction(equation="H + OH = H2O", rate=ct.Arrhenius(0.0, 0.0, 0.0))
gas.add_reaction(R1)
gas.add_reaction(R2)

In [ ]:
iR1 = gas.n_reactions - 2
iR2 = gas.n_reactions - 1

T = np.linspace(300, 2500, 500)
states = ct.SolutionArray(gas, len(T))
states.TPX = T, ct.one_atm, "CO2:1.0, H2O:1.0, N2:3.76, O2:1.0"
dh0 = states.delta_standard_enthalpy[:, [iR1,iR2]]
ds0 = states.delta_standard_entropy[:, [iR1,iR2]]
Kc = states.equilibrium_constants[:, [iR1,iR2]]
Kc.shape

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(T, dh0)
ax[1].plot(T, ds0)
ax[0].legend([R1.equation, R2.equation])

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(T, Kc)